# Terrain Classification - Combined User Data
### Created by Keenan McConkey 2019.08.01

In [1]:
from __future__ import absolute_import, division, print_function

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import pymrmr
import sklearn

## Part 1 - Importing Preprocessed Data

### Part (a) - Functions for Data Import

In [2]:
# All the terrains, placements, vectors, power-assistance, users in the study
terrains = ['Concrete', 'Carpet', 'Linoleum', 'Asphalt', 'Sidewalk', 'Grass', 'Gravel']
powers = ['Manual'] # TODO: Fix power PSD data and add back in
placements_manual = ['Middle', 'Left', 'Right', 'Synthesis']
placements_power = ['Middle']
vectors = ['TimeFeats', 'FreqFeats', 'FFTs', 'PSDLogs']
users = ['All']
axes = ['X Accel', 'Y Accel', 'Z Accel', 'X Gyro', 'Y Gyro', 'Z Gyro']

In [3]:
'''Get the integer terrain value of a given label'''
def get_terrain_num(_label):
    for i, terrain in enumerate(terrains):
        if terrain in _label:
            return i
        
    raise Exception('Unknown terrain')

'''Get the name associated with a terrain integer'''
def get_terrain_name(terrain_num):
    return terrains[terrain_num]
    return terrains[terrain_num]

'''Get the placement location name for given label'''
def get_placement(_label):
    for placement in placements:
        if placement in _label:
            return placement
    
    raise Exception('Unknown placement')

'''Get the transform used for given label'''
def get_transform(_label):
    for transform in transforms:
        if transform in _label:
            return transform
    
    raise Exception('Unknown transform')

### Part (b) - Import Processed Data from Each User

In [4]:
'''Combine data from labelled datasets into a single dataframe'''
def combine_datasets(datasets):
    return pd.concat(list(datasets.values()), ignore_index=True, sort=False)

In [7]:
path = 'processed_data/new_setup/new_columning/' 

# Nested dictionary of processed data:
# - Power assistance type
# -- Placement
# --- Feature Vector
# ---- User
power_dict = {}

# Create each nesting of the dictionary
for power in powers:
    placement_dict = {}
    
    # Power datasets only have middle placement (for now)
    if power == 'Power':
        placements = placements_power.copy()
    else:
        placements = placements_manual.copy()
    
    for placement in placements:
        vector_dict = {}

        for vector in vectors:
            user_dict = {}

            for user in users:
                # File name based on above parameters
                filename = power.lower() + '/' + placement + '_' + vector + '_' + user 
                if power == 'Power':
                    filename += '_Power'
                filename += '.csv'
                
                # Read data and update current user dictionary
                data = pd.read_csv(path + filename)
                user_dict.update({user: data})

            # Combine users to form a new entry of user dictionary, save to .csv
            # NaNs arise when you combine Synthesis feature vectors
            #combined_data = combine_datasets(user_dict).dropna(axis='columns')
            #user_dict.update({'All': combined_data})

            vector_dict.update({vector: user_dict})
        
        # Create a dictionary of the combined feature vector for each user
        combined_vector_user_dict = {}
        
        for user in user_dict.keys():
            # Get all vectors for current user and pop label column
            user_all_vectors = []
            
            for vector in vector_dict.values():
                user_vector = vector[user].copy()
                labels = user_vector.pop('Label') # All label columns should be the same
                user_all_vectors.append(user_vector)
            
            # Combine vectors and add back label column
            combined_vector = pd.concat(user_all_vectors, axis='columns')
            combined_vector.insert(loc=0, column='Label', value=labels)
            combined_vector_user_dict.update({user: combined_vector})
        
        # Add the combined feature vector to the vector dictionary
        vector_dict.update({'Combined': combined_vector_user_dict})
        
        placement_dict.update({placement: vector_dict})
    
    power_dict.update({power: placement_dict})

In [8]:
# Update vectors to reflect new combined vector
vectors.append('Combined')

In [9]:
# Check some data
power_dict['Manual']['Middle']['Combined']['All'].tail()

,Label,Mean X Accel Middle,Std X Accel Middle,Norm X Accel Middle,AC X Accel Middle,Max X Accel Middle,Min X Accel Middle,RMS X Accel Middle,ZCR X Accel Middle,Skew X Accel Middle,...,PSDLog 50 Hz Z Gyro Middle,PSDLog 51 Hz Z Gyro Middle,PSDLog 52 Hz Z Gyro Middle,PSDLog 53 Hz Z Gyro Middle,PSDLog 54 Hz Z Gyro Middle,PSDLog 55 Hz Z Gyro Middle,PSDLog 56 Hz Z Gyro Middle,PSDLog 57 Hz Z Gyro Middle,PSDLog 58 Hz Z Gyro Middle,PSDLog 59 Hz Z Gyro Middle
16065,4,-2.993490,0.058487,0.409874,0.094989,-0.334863,-0.373391,0.409337,0.086221,0.142740,...,0.675446,0.980595,1.012408,0.896429,1.043201,0.868404,1.153671,1.019332,1.010432,0.885506
16066,4,-5.150751,-0.420820,0.824175,0.433643,-0.860113,-0.373391,0.823611,-1.417281,-0.250344,...,1.898609,1.801497,1.839670,1.786389,1.775234,1.902943,1.784176,1.816177,1.817222,1.786705
16067,4,-2.704405,-0.045126,0.249784,-0.019371,-0.793978,-0.047606,0.249256,-0.414946,-0.222345,...,0.912860,0.935080,1.081082,1.002101,0.969530,1.040619,1.045307,1.018618,0.919208,0.989472
16068,4,2.020397,-0.459843,-0.191738,-0.287109,-0.497873,0.697808,-0.192238,-0.359261,-0.638358,...,1.552285,1.499390,1.544513,1.482069,1.469938,1.493814,1.545534,1.521183,1.503133,1.538029
16069,4,2.414125,0.033483,0.312220,0.024136,-0.173717,0.378144,0.311688,-0.971799,-0.985338,...,1.394192,1.466845,1.494007,1.416393,1.453440,1.400857,1.398331,1.511435,1.459463,1.516934


## Part 2 -  mRMR (minimum Redunancy Maximum Relevance)

mRMR tries to find which features have the highest information shared with classified state and lowest information shared with other features.

### Part (a) - Middle Frame Placement

#### Part (i) - Manual Wheelchair

In [10]:
pymrmr.mRMR(data=power_dict['Manual']['Middle']['Features']['All'], method='MID', nfeats=5)

['FC Y Gyro Middle',
 'ZCR Y Accel Middle',
 'ZCR Z Accel Middle',
 'VF X Gyro Middle',
 'VF Y Gyro Middle']

In [11]:
pymrmr.mRMR(data=power_dict['Manual']['Middle']['FFTs']['All'], method='MID', nfeats=5)

['FFT 14 Hz Y Accel Middle',
 'FFT 1 Hz Y Gyro Middle',
 'FFT 2 Hz X Gyro Middle',
 'FFT 17 Hz Z Accel Middle',
 'FFT 21 Hz Y Accel Middle']

In [12]:
pymrmr.mRMR(data=power_dict['Manual']['Middle']['PSDLogs']['All'], method='MID', nfeats=5)

['PSDLog 33 Hz Y Gyro Middle',
 'PSDLog 0 Hz Y Gyro Middle',
 'PSDLog 56 Hz Y Accel Middle',
 'PSDLog 1 Hz X Gyro Middle',
 'PSDLog 59 Hz Y Accel Middle']

In [13]:
pymrmr.mRMR(data=power_dict['Manual']['Middle']['Combined']['All'], method='MID', nfeats=5)

['FC Y Gyro Middle',
 'PSDLog 12 Hz Z Accel Middle',
 'VF X Gyro Middle',
 'ZCR Z Accel Middle',
 'ZCR Y Accel Middle']

#### Part (i) - Power Assist Wheelchair

In [14]:
#pymrmr.mRMR(data=power_dict['Power']['Middle']['Features']['All'], method='MID', nfeats=5)

In [15]:
#pymrmr.mRMR(data=power_dict['Power']['Middle']['FFTs']['All'], method='MID', nfeats=5)

In [16]:
#pymrmr.mRMR(data=power_dict['Power']['Middle']['PSDLogs']['All'], method='MID', nfeats=5)

In [17]:
#pymrmr.mRMR(data=power_dict['Power']['Middle']['Combined']['All'], method='MID', nfeats=5)

### Part (b) - Left Wheel Placement

In [18]:
pymrmr.mRMR(data=power_dict['Manual']['Left']['Features']['All'], method='MID', nfeats=5)

['Std Z Accel Left',
 'FC Z Accel Left',
 'ZCR Y Gyro Left',
 'ZCR Z Accel Left',
 'Min Z Gyro Left']

In [19]:
pymrmr.mRMR(data=power_dict['Manual']['Left']['FFTs']['All'], method='MID', nfeats=5)

['FFT 20 Hz Z Accel Left',
 'FFT 43 Hz Z Accel Left',
 'FFT 14 Hz Z Accel Left',
 'FFT 63 Hz Z Accel Left',
 'FFT 38 Hz Z Accel Left']

In [20]:
pymrmr.mRMR(data=power_dict['Manual']['Left']['PSDLogs']['All'], method='MID', nfeats=5)

['PSDLog 27 Hz Z Accel Left',
 'PSDLog 62 Hz Z Accel Left',
 'PSDLog 17 Hz Z Accel Left',
 'PSDLog 40 Hz Z Accel Left',
 'PSDLog 48 Hz X Accel Left']

In [21]:
pymrmr.mRMR(data=power_dict['Manual']['Left']['Combined']['All'], method='MID', nfeats=5)

['PSDLog 27 Hz Z Accel Left',
 'FC Z Accel Left',
 'PSDLog 17 Hz Z Accel Left',
 'PSDLog 63 Hz Z Accel Left',
 'PSDLog 48 Hz X Accel Left']

### Part (c) - Right Wheel Placement

In [22]:
pymrmr.mRMR(data=power_dict['Manual']['Right']['Features']['All'], method='MID', nfeats=5)

['ZCR Z Accel Right',
 'RMS Z Accel Right',
 'VF Z Accel Right',
 'Max Z Gyro Right',
 'FC X Accel Right']

In [23]:
pymrmr.mRMR(data=power_dict['Manual']['Right']['FFTs']['All'], method='MID', nfeats=5)

['FFT 24 Hz Z Accel Right',
 'FFT 63 Hz Z Accel Right',
 'FFT 18 Hz Z Accel Right',
 'FFT 14 Hz Z Accel Right',
 'FFT 53 Hz Z Accel Right']

In [24]:
pymrmr.mRMR(data=power_dict['Manual']['Right']['PSDLogs']['All'], method='MID', nfeats=5)

['PSDLog 62 Hz Z Accel Right',
 'PSDLog 15 Hz Z Accel Right',
 'PSDLog 55 Hz Z Accel Right',
 'PSDLog 25 Hz Z Accel Right',
 'PSDLog 63 Hz Z Accel Right']

In [25]:
pymrmr.mRMR(data=power_dict['Manual']['Right']['Combined']['All'], method='MID', nfeats=5)

['ZCR Z Accel Right',
 'PSDLog 16 Hz Z Accel Right',
 'VF Z Accel Right',
 'PSDLog 26 Hz Z Accel Right',
 'PSDLog 62 Hz Z Accel Right']

### Part (d) - Synthesis "Placement"

In [26]:
pymrmr.mRMR(data=power_dict['Manual']['Synthesis']['Features']['All'], method='MID', nfeats=5)

['ZCR Calc X Accel Synthesis',
 'VF Left XY Accel Synthesis',
 'ZCR Right Z Accel Synthesis',
 'Min Left XY Accel Synthesis',
 'FC Right XY Accel Synthesis']

In [27]:
pymrmr.mRMR(data=power_dict['Manual']['Synthesis']['FFTs']['All'], method='MID', nfeats=5)

['FFT 29 Hz Left XY Accel Synthesis',
 'FFT 0 Hz Calc X Vel Synthesis',
 'FFT 21 Hz Right Z Accel Synthesis',
 'FFT 27 Hz Right XY Accel Synthesis',
 'FFT 58 Hz Left XY Accel Synthesis']

In [28]:
pymrmr.mRMR(data=power_dict['Manual']['Synthesis']['PSDLogs']['All'], method='MID', nfeats=5)

['PSDLog 35 Hz Left XY Accel Synthesis',
 'PSDLog 15 Hz Right Z Accel Synthesis',
 'PSDLog 55 Hz Right XY Accel Synthesis',
 'PSDLog 55 Hz Left XY Accel Synthesis',
 'PSDLog 62 Hz Right Z Accel Synthesis']

In [29]:
pymrmr.mRMR(data=power_dict['Manual']['Synthesis']['Combined']['All'], method='MID', nfeats=5)

['ZCR Calc X Accel Synthesis',
 'VF Left XY Accel Synthesis',
 'ZCR Right Z Accel Synthesis',
 'Min Left XY Accel Synthesis',
 'FC Right XY Accel Synthesis']

### Part (e) - Nested Dictionary of Top Features

In [ ]:
# Create dictionary of top mRMR features to speed up calculations, up to to 20
power_dict_features = {}

for power in powers:
    placement_dict_features = {}
    
    for placement in placements:
        vector_dict_features = {}
        
        for vector in vectors:
            user_dict_features = {}

            # Only calculate for all users for now
            for user in ['All']:
                top_features = pymrmr.mRMR(data=power_dict[power][placement][vector][user],
                                           method='MID', nfeats=20)
                user_dict_features.update({user: top_features})

            vector_dict_features.update({vector: user_dict_features})
            
        placement_dict_features.update({placement: vector_dict_features})
    
    power_dict_features.update({power: placement_dict_features})\

In [ ]:
# Pickle the dictionary
import pickle

mRMR_dict_filename = '/home/caris/Wheelchair-Terrain-Classification/dicts/mRMR_Dictionary.pkl'
outfile = open(mRMR_dict_filename, 'wb')
pickle.dump(power_dict_features, outfile)
outfile.close()

In [ ]:
# Unpickle the dictionary
#infile = open(mRMR_dict_filename, 'rb')
#power_dict_features = pickle.load(infile)
#infile.close()

## Part 3 - PCA (Principal Component Analysis)

In [ ]:
from sklearn.decomposition import PCA

def get_pca_df(combined_data, n_components=2):
    # Setup PCA parameters
    pca = PCA(n_components=n_components)
    
    # Copy data to avoid modification
    data = combined_data.copy()
    
    # Extract terrain labels
    labels = data.pop('Label')
    
    # Get specified number of principal components and convert to dataframe
    pc = pca.fit_transform(data)
    pc_df = pd.DataFrame(data=pc, columns=['PC {}'.format(i + 1) for i in range(n_components)])
    
    #print('Explained Variance of Each PC: {}'.format(pca.explained_variance_ratio_))
    #print('Total Explained Variance: {}'.format(np.sum(pca.explained_variance_ratio_)))
    
    return pd.concat([labels, pc_df], axis='columns')

In [ ]:
get_pca_df(power_dict['Manual']['Synthesis']['Combined']['All']).tail()

In [ ]:
get_pca_df(power_dict['Manual']['Synthesis']['Combined']['All'], n_components=5).tail()

In [ ]:
def visualize_2d_pca(pca_2d_df, figsize=(8, 8)):
    # Plot parameters
    plt.clf()
    plt.figure(figsize=figsize)
    plt.xlabel('Principal Component 1')
    plt.ylabel('Principal Component 2')
    
    # Scatter plot of each terrain
    for terrain in terrains:
        terrain_indices = pca_2d_df['Label'] == get_terrain_num(terrain)
        plt.scatter(pca_2d_df.loc[terrain_indices, 'PC 1'], pca_2d_df.loc[terrain_indices, 'PC 2'])
    
    plt.legend(terrains)
    plt.show()

In [ ]:
visualize_2d_pca(get_pca_df(power_dict['Manual']['Right']['Features']['All']))

## Part 4 - Comparing Classifier Accuracy

In [ ]:
from sklearn.model_selection import KFold

'''Run train test k-fold times
   Returns an tuple of arrays, where arrays elements are actual/predicted labels for each k-fold test'''
def train_test_k_fold(combined_data, n_splits, model):
    # Shuffle ensures we get a mix of terrains
    kf = KFold(n_splits=n_splits, shuffle=True)

    # Copy data to avoid modification
    data = combined_data.copy()
    
    # Extract terrain labels
    labels = data.pop('Label')

    # Array of predicted labels for each k fold
    test_k_fold = []
    predict_k_fold = []

    # Split into n splits
    for train_index, test_index in kf.split(data):
        train, test = data.loc[train_index], data.loc[test_index]
        train_labels, test_labels = labels.loc[train_index], labels.loc[test_index]
        
        # Actual labels
        test_k_fold.append(test_labels)
        
        # Train and test model
        model.fit(train, train_labels)
        predict_k_fold.append(model.predict(test))

    return (test_k_fold, predict_k_fold)

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Dictionary of classifiers
classifiers = {'Naive Bayes': GaussianNB(),
               'k Nearest': KNeighborsClassifier(),
               'Decision Tree': DecisionTreeClassifier(), 
               'Random Forest': RandomForestClassifier(n_estimators=100),
               'AdaBoost': AdaBoostClassifier(),
               'Support Vector Machine': SVC(gamma='scale')}

In [ ]:
def create_accuracy_table(n_splits, power_type='Manual', user_name='All', separate_axes=False,
                          feat_selection='None', n_feats=None):
    
    # Power type affects which placements are available
    if power_type == 'Manual':
        placements = placements_manual
    else:
        placements = placements_power
    
    # Dataframe table of accuracies for each classifier for each placement
    if separate_axes:
        vector_indices = [p + ' ' + v + ' ' + a for p in placements for v in vectors for a in axes]
    else:
        vector_indices = [p + ' ' + v for p in placements for v in vectors]
    
    # Vector column contains each vector for each placement and axes
    accuracy_table = pd.DataFrame({'Vector': vector_indices})

    # Calculate accuracy for each placement for each feature vector and classifier
    for classifier_name, classifier in classifiers.items():
        model = classifier

        # Row dictionary for given model
        rows = {}

        # Add current classifier to row dictionary
        for placement in placements:
            for vector in vectors:
                # Extract data for above parameters
                data = power_dict[power_type][placement][vector][user_name].copy()
                
                # Iterate through axes if we want them separate, else just put in empty string
                if separate_axes:
                    itr_axes = axes
                else:
                    itr_axes = ['All']
                
                for axis in itr_axes:
                    index_name = placement + ' ' + vector
                    data_temp = data
                    
                    # If separate axes, get data with only columns matching current axes
                    if separate_axes:
                        index_name += ' ' + axis
                        matching_columns = [column for column in data.columns if axis in column]
                        matching_columns.insert(0, 'Label')
                        data_temp = data_temp[matching_columns]
                    
                    # Use only the top features using mRMR feature selection
                    if feat_selection == 'mRMR':
                        top_feats = power_dict_features[power_type][placement][vector][user_name].copy()
                        top_feats.insert(0, 'Label')
                        data_temp = data_temp[top_feats[:n_feats+1]]

                    # Run PCA on the data
                    elif feat_selection == 'PCA':
                        data_temp = get_pca_df(data_temp, n_components=n_feats)

                    # Extract predicted and actual labels for requested user
                    actual, predict = train_test_k_fold(data_temp, n_splits, model)

                    # Take mean accuracy of k fold testing
                    accuracies = []
                    for i in range(len(predict)):
                        accuracies.append(accuracy_score(actual[i], predict[i]))
                    
                    rows.update({index_name: np.mean(accuracies)})

        # Update accuracy table with classifier column by mapping row names to indices
        accuracy_table[classifier_name] = accuracy_table['Vector'].map(rows)
    
    return accuracy_table

### Part (a) - Combined Axes Accuracy Table
#### Part (i) - No Feature Selection

In [ ]:
# Create accuracy table for 5 k-fold splits without any feature selection
accuracy_table = create_accuracy_table(n_splits=5, power_type='Manual', user_name='All')

In [ ]:
accuracy_table

#### Part (ii) - mRMR Feature Selection

In [ ]:
# Create accuracy table for 5 k-fold splits with mRMR feature selection
accuracy_table_mRMR = create_accuracy_table(n_splits=5, power_type='Manual', user_name='All',
                                            feat_selection='mRMR', n_feats=20)

In [ ]:
accuracy_table_mRMR

#### Part (iii) - PCA Feature Selection

In [ ]:
# Create accuracy table for 5 k-fold splits with mRMR feature selection
accuracy_table_PCA = create_accuracy_table(n_splits=5, power_type='Manual', user_name='All',
                                            feat_selection='PCA', n_feats=20)

In [ ]:
accuracy_table_PCA

### Part (b) - Separate Axes Accuracy Table

#### Part (i) - mRMR Feature Selection

In [ ]:
# Create accuracy table for 5 k-fold splits with mRMR feature selection
accuracy_table_sep_mRMR = create_accuracy_table(n_splits=5, power_type='Manual', user_name='All',
                                            separate_axes=True, feat_selection='PCA', n_feats=10)

In [ ]:
accuracy_table_sep_mRMR

### Glossary

`Dataset` - Batch of data recorded on one terrain type

`Data Window` - Split up portion of a `Dataset`

`Direction / Axes` - Linear acceleration or gyroscope in $x,y$ or $z$

`Feature Vector` - Any feature of the data that can be used to classify terrain, e.g. Z Accel Mean, Y Accel FFT, etc

`Extracted Feature Vector` - Features that aren't from transforms, e.g. Z Accel Min, Y Accel Autocorrelation, etc

`Placement` - One of three IMU placements on the wheelchair, i.e. Middle, Left, or Right